<a href="https://colab.research.google.com/github/Yuvraj-Jaiswal/Vgg16-on-Chest-CT-scan/blob/main/Vgg16_Covid_CT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Flatten
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.python.keras.utils import data_utils
from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import VGG16

Shape = 224

WEIGHTS_PATH_NO_TOP = ('https://storage.googleapis.com/tensorflow/'
                       'keras-applications/vgg16/'
                       'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')

Data_Generator = ImageDataGenerator(rotation_range=30,width_shift_range=0.1,
                                    height_shift_range=0.1,shear_range=0.2,zoom_range=0.2,fill_mode='nearest'
                                    ,horizontal_flip=False,rescale=1/255)

train_data = Data_Generator.flow_from_directory(directory="/content/drive/MyDrive/Data/Data/Train",
                                             target_size=(Shape,Shape),batch_size=16,class_mode = 'binary')

test_data = Data_Generator.flow_from_directory(directory="/content/drive/MyDrive/Data/Data/Test",
                                               target_size=(Shape,Shape),batch_size=16,class_mode = 'binary')

CheckpointSaver = ModelCheckpoint("Checkpoints\\model.{epoch:02d}-{val_accuracy:.2f}.h5",monitor='val_accuracy',mode='max',save_best_only=True,verbose=1)
Tensorboard_loger = TensorBoard(log_dir='./logs')

model = Sequential()

model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=(Shape,Shape,3),activation='relu',padding='same'))
model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu',padding='same'))
model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding='same'))
model.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding='same'))
model.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same'))
model.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same'))
model.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same'))
model.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same'))
model.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

weights_path = data_utils.get_file(
          'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
          WEIGHTS_PATH_NO_TOP,
          cache_subdir='models',
          file_hash='6d6bbae143d832006294945121d1f1fc')

model.load_weights(weights_path)

for layer in model.layers:
  layer.trainable = False

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(train_data,epochs=400,validation_data=test_data,callbacks=[CheckpointSaver,Tensorboard_loger],workers=10)

Found 2281 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Epoch 1/400
143/143 [==============================] - ETA: 0s - loss: 0.4555 - accuracy: 0.7795
Epoch 1: val_accuracy improved from -inf to 0.69000, saving model to Checkpoints\model.01-0.69.h5
143/143 [==============================] - 54s 352ms/step - loss: 0.4555 - accuracy: 0.7795 - val_loss: 0.5804 - val_accuracy: 0.6900
Epoch 2/400
143/143 [==============================] - ETA: 0s - loss: 0.3379 - accuracy: 0.8536
Epoch 2: val_accuracy did not improve from 0.69000
143/143 [==============================] - 45s 303ms/step - loss: 0.3379 - accuracy: 0.8536 - val_loss: 0.6792 - val_accuracy: 0.6850
Epoch 3/400
143/143 [==============================] - ETA: 0s - loss: 0.2829 - accuracy: 0.8829
Epoch 3: val_accuracy improved from 0.69000 to 0.78000, saving model to Checkpoints\model.03-0.78.h5
143/143 [==============================] - 50s 321ms/step - loss: 0.2829 - accuracy: 0.8829 - val_loss: 0.45

KeyboardInterrupt: ignored